# Генерация текста на основе "Гарри Поттер и методы рационального мышления"

## Описание проекта

Данный проект реализует различные подходы к **генерации текста** с использованием современных методов обработки естественного языка. В качестве обучающих данных используется фанфик "Гарри Поттер и методы рационального мышления" - произведение, сочетающее магический мир с научным подходом к мышлению.

### Основные задачи:
- **Сбор и предобработка данных**: Извлечение текста с веб-сайта и очистка
- **Различные типы токенизации**: Посимвольная, пословная, BPE
- **Архитектуры нейронных сетей**: RNN, LSTM, Bidirectional LSTM, Transformer
- **Генерация текста**: Создание новых текстов в стиле оригинала

### Изучаемые модели:
1. **Simple RNN** - базовая рекуррентная сеть
2. **LSTM** - долгосрочная память
3. **Bidirectional LSTM** - двунаправленная обработка
4. **GPT-style Transformer** - архитектура внимания

### Начнем с загрузки и предобработки данных:


## Сбор данных с веб-сайта

### Загрузка текста из интернета


In [ ]:
# Импорт необходимых библиотек для веб-скрапинга
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

# Базовый URL для загрузки глав
base_url = "https://hpmor.ru/book/1/"
all_text = ""

print("Начинаем загрузку текста из интернета...")
print("Источник: https://hpmor.ru/book/1/")

# Загружаем первые 10 глав для демонстрации
for chapter in tqdm(range(1, 11), desc="Загрузка глав"):
    url = f"{base_url}{chapter}/"
    try:
        # Отправляем GET-запрос
        response = requests.get(url)
        response.raise_for_status()  # Проверяем успешность запроса

        # Парсим HTML-содержимое
        soup = BeautifulSoup(response.content, "html.parser")
        paragraphs = soup.find_all("p")

        # Ищем индекс абзаца со звездочками (разделитель между метаданными и текстом)
        start_index = next(
            (i for i, p in enumerate(paragraphs) if p.get_text(strip=True) == "* * *"),
            None
        )

        if start_index is None:
            print(f"Глава {chapter}: звездочки не найдены, пропускаем")
            continue

        # Берем только абзацы после звездочек и исключаем теги <em>
        filtered_paragraphs = [
            p.get_text(strip=True) for p in paragraphs[start_index + 1:]
            if not p.find("em")  # Исключаем курсивные примечания
        ]

        chapter_text = "\n".join(filtered_paragraphs)

        # Добавляем заголовок главы и текст
        all_text += f"\n\n=== Глава {chapter} ===\n\n"
        all_text += chapter_text

        # Вежливая задержка между запросами
        time.sleep(0.2)

    except Exception as e:
        print(f"Глава {chapter}: ошибка — {e}")

# Сохраняем результат в файл
with open("harry.txt", "w", encoding="utf-8") as f:
    f.write(all_text)

print("Загрузка завершена! Текст сохранен в файл: harry.txt")
print(f"Общий размер текста: {len(all_text)} символов")


## Предобработка текста

### Функции очистки и нормализации


In [ ]:
# Импорт библиотек для обработки текста
import pandas as pd
import re

def remove_dialog_dashes(text: str) -> str:
    """
    Удаляет тире в начале строк (реплики диалога)
    
    Args:
        text (str): Исходный текст
        
    Returns:
        str: Текст без тире в начале строк
    """
    # Удаляем тире, дефисы и длинные тире в начале строк
    return re.sub(r"(?m)^[\s]*[–—-]\s*", "", text)

def keep_only_russian(text: str) -> str:
    """
    Оставляет только русские буквы и базовые знаки препинания
    
    Args:
        text (str): Исходный текст
        
    Returns:
        str: Очищенный текст в нижнем регистре
    """
    # Приводим к нижнему регистру
    text = text.lower()
    
    # Оставляем только русские буквы, пробелы и основные знаки препинания
    text = re.sub(r"[^а-яё .,!?;\n]", " ", text)
    
    # Убираем лишние пробелы
    text = re.sub(r"\s+", " ", text)
    
    return text.strip()

print("Функции предобработки текста определены:")
print("- remove_dialog_dashes(): удаление тире из диалогов")
print("- keep_only_russian(): нормализация русского текста")


In [ ]:
# Загружаем сохраненный текст
with open("harry.txt", "r", encoding="utf-8") as f:
    all_text = f.read()

print(f"Загружен текст размером: {len(all_text)} символов")

# Разделяем главы по заголовкам
chapters_raw = re.split(r"\n+=+ Глава (\d+) =+\n+", all_text)

chapters = []
for i in range(1, len(chapters_raw), 2):
    chapter_number = int(chapters_raw[i])
    chapter_text = chapters_raw[i + 1].strip()

    # Применяем функции очистки
    chapter_text = remove_dialog_dashes(chapter_text)
    chapter_text = keep_only_russian(chapter_text)

    chapters.append({"chapter": chapter_number, "text": chapter_text})

# Создаем DataFrame для удобной работы с данными
df = pd.DataFrame(chapters)
df = df.sort_values("chapter").reset_index(drop=True)

print(f"Обработано глав: {len(df)}")
print("\nПримеры обработанных глав:")
print(df.head())

# Объединяем весь очищенный текст
full_cleaned_text = " ".join(df["text"].astype(str).tolist())
print(f"\nОбщий размер очищенного текста: {len(full_cleaned_text)} символов")


# Часть 1: Simple RNN

## Посимвольная токенизация


In [ ]:
# Импорт библиотек для глубокого обучения
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

print("Настраиваем Simple RNN для посимвольной генерации текста...")

# 1. Подготовка алфавита
chars = sorted(set(full_cleaned_text))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for c, i in char2idx.items()}
vocab_size = len(chars)

print(f"Размер словаря символов: {vocab_size}")
print(f"Первые 20 символов: {chars[:20]}")

# 2. Создание обучающих данных
seq_length = 40  # Длина входной последовательности
step = 3         # Шаг скользящего окна
input_seqs = []
target_chars = []

print("Создаем обучающие последовательности...")

for i in range(0, len(full_cleaned_text) - seq_length, step):
    input_seq = full_cleaned_text[i:i + seq_length]
    target_char = full_cleaned_text[i + seq_length]
    input_seqs.append([char2idx[c] for c in input_seq])
    target_chars.append(char2idx[target_char])

print(f"Создано обучающих последовательностей: {len(input_seqs)}")

# 3. One-hot encoding
X = tf.keras.utils.to_categorical(input_seqs, num_classes=vocab_size)
y = tf.keras.utils.to_categorical(target_chars, num_classes=vocab_size)

print(f"Размер входных данных: {X.shape}")
print(f"Размер целевых данных: {y.shape}")

# 4. Создание модели Simple RNN
model = Sequential([
    SimpleRNN(128, input_shape=(seq_length, vocab_size)),
    Dense(vocab_size, activation='softmax')
])

print("Архитектура модели:")
model.summary()

# Компиляция модели
model.compile(loss='categorical_crossentropy', optimizer='adam')
print("Модель скомпилирована и готова к обучению!")


In [ ]:
# Обучение модели Simple RNN
print("Начинаем обучение Simple RNN...")
print("Это может занять некоторое время...")

# Обучение модели
model.fit(X, y, batch_size=128, epochs=200)

print("Обучение завершено!")


In [ ]:
def generate_text(seed_text, gen_length=300):
    """
    Функция для генерации текста с помощью обученной модели
    
    Args:
        seed_text (str): Начальный текст для генерации
        gen_length (int): Длина генерируемого текста
        
    Returns:
        str: Сгенерированный текст
    """
    generated = seed_text
    
    for _ in range(gen_length):
        # Подготавливаем входную последовательность
        input_seq = [char2idx.get(c, 0) for c in generated[-seq_length:]]
        input_seq = tf.keras.utils.to_categorical(input_seq, num_classes=vocab_size)
        input_seq = np.expand_dims(input_seq, axis=0)  # Добавляем размерность батча

        # Получаем предсказание следующего символа
        prediction = model.predict(input_seq, verbose=0)[0]
        
        # Выбираем следующий символ на основе вероятностей
        next_index = np.random.choice(range(vocab_size), p=prediction)
        next_char = idx2char[next_index]

        generated += next_char
    
    return generated

# Тестируем генерацию текста
print("Генерируем текст на основе слова 'язык':")
generated_text1 = generate_text("язык")
print(generated_text1)

print("\nГенерируем текст на основе слова 'зелье':")
generated_text2 = generate_text("зелье")
print(generated_text2)


## Пословная токенизация


In [ ]:
# Настройка Simple RNN для пословной генерации
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("Настраиваем Simple RNN для пословной генерации...")

# 1. Подготовка: токенизация текста на слова
tokenizer = Tokenizer()
tokenizer.fit_on_texts([full_cleaned_text])

word_index = tokenizer.word_index          # Слово → индекс
index_word = {v: k for k, v in word_index.items()}  # Индекс → слово
total_words = len(word_index) + 1          # Добавляем 1 для padding

print(f"Всего уникальных слов: {total_words}")
print(f"Первые 10 слов в словаре: {list(word_index.keys())[:10]}")

# 2. Создание обучающих последовательностей (n-граммы)
input_sequences = []
words = full_cleaned_text.split()

# Скользящее окно для генерации n-грамм
window_size = 5
for i in range(1, len(words)):
    seq = words[max(0, i - window_size):i + 1]
    encoded = tokenizer.texts_to_sequences([" ".join(seq)])[0]
    if len(encoded) >= 2:
        input_sequences.append(encoded)

print(f"Количество обучающих последовательностей: {len(input_sequences)}")

# 3. Подготовка X и y
max_seq_len = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len)

X_words = input_sequences[:, :-1]  # все слова, кроме последнего
y_words = tf.keras.utils.to_categorical(input_sequences[:, -1], num_classes=total_words)

print(f"Размер входных данных: {X_words.shape}")
print(f"Размер целевых данных: {y_words.shape}")

# 4. Построение модели Simple RNN для слов
model_word = Sequential([
    tf.keras.layers.Embedding(input_dim=total_words, output_dim=64, input_length=X_words.shape[1]),
    SimpleRNN(128),
    Dense(total_words, activation='softmax')
])

model_word.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Архитектура модели для слов:")
model_word.summary()


# Часть 2: LSTM (Long Short-Term Memory)

## Однонаправленная LSTM


### Посимвольная токенизация с LSTM


In [ ]:
# Импорт библиотек для LSTM
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

print("Настраиваем LSTM для посимвольной генерации...")

# Подготовка данных (аналогично Simple RNN)
chars = sorted(set(full_cleaned_text))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for c, i in char2idx.items()}
vocab_size = len(chars)

print(f"Размер словаря символов: {vocab_size}")

# Создание последовательностей
seq_length = 40
step = 3
sequences = []
next_chars = []

for i in range(0, len(full_cleaned_text) - seq_length, step):
    seq = full_cleaned_text[i:i+seq_length]
    target = full_cleaned_text[i+seq_length]
    sequences.append([char2idx[c] for c in seq])
    next_chars.append(char2idx[target])

# Преобразование в one-hot encoding
X = to_categorical(sequences, num_classes=vocab_size)
y = to_categorical(next_chars, num_classes=vocab_size)

print(f"Размер обучающих данных: {X.shape}")
print(f"Размер целевых данных: {y.shape}")

# Создание модели LSTM
model_lstm_1 = Sequential([
    LSTM(128, input_shape=(seq_length, vocab_size)),
    Dense(vocab_size, activation='softmax')
])

model_lstm_1.compile(loss='categorical_crossentropy', optimizer='adam')

print("Архитектура однослойной LSTM:")
model_lstm_1.summary()


In [ ]:
# Обучение однослойной LSTM
print("Начинаем обучение однослойной LSTM...")

model_lstm_1.fit(X, y, batch_size=128, epochs=50)

print("Обучение однослойной LSTM завершено!")


In [ ]:
def generate_text_char_lstm(model, seed_text="гарри", length=20, temperature=1.0):
    """
    Функция генерации текста с использованием LSTM
    
    Args:
        model: Обученная LSTM модель
        seed_text (str): Начальный текст
        length (int): Длина генерируемого текста
        temperature (float): Температура для сэмплирования
        
    Returns:
        str: Сгенерированный текст
    """
    generated = seed_text
    
    for _ in range(length):
        # Подготавливаем входную последовательность
        input_seq = [char2idx.get(c, 0) for c in generated[-seq_length:]]
        input_seq = tf.keras.utils.to_categorical(input_seq, num_classes=vocab_size)
        input_seq = np.expand_dims(input_seq, axis=0)

        # Получаем предсказания
        preds = model.predict(input_seq, verbose=0)[0]
        preds = np.asarray(preds).astype("float64")
        
        # Применяем температуру для контроля случайности
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        # Выбираем следующий символ
        next_idx = np.random.choice(range(vocab_size), p=preds)
        next_char = idx2char[next_idx]
        generated += next_char
    
    return generated

# Тестируем генерацию с однослойной LSTM
print("Генерация текста с однослойной LSTM:")
print("Начальное слово: 'гермиона'")
generated_text = generate_text_char_lstm(model_lstm_1, "гермиона")
print(generated_text)

print("\nГенерация с другим начальным словом:")
print("Начальное слово: 'пожиратели'")
generated_text2 = generate_text_char_lstm(model_lstm_1, "пожиратели")
print(generated_text2)


### Многослойная LSTM


In [ ]:
# Создание многослойной LSTM модели
print("Создаем многослойную LSTM модель...")

model_lstm_multi = Sequential([
    LSTM(128, return_sequences=True, input_shape=(seq_length, vocab_size)),
    LSTM(128),
    Dense(vocab_size, activation='softmax')
])

model_lstm_multi.compile(loss='categorical_crossentropy', optimizer='adam')

print("Архитектура многослойной LSTM:")
model_lstm_multi.summary()

# Обучение многослойной LSTM
print("Начинаем обучение многослойной LSTM...")
model_lstm_multi.fit(X, y, batch_size=128, epochs=50)

print("Обучение многослойной LSTM завершено!")

# Тестируем многослойную LSTM
print("\nГенерация текста с многослойной LSTM:")
print("Начальное слово: 'драко'")
generated_multi = generate_text_char_lstm(model_lstm_multi, "драко")
print(generated_multi)

print("\nГенерация с другим начальным словом:")
print("Начальное слово: 'поттер'")
generated_multi2 = generate_text_char_lstm(model_lstm_multi, "поттер")
print(generated_multi2)


### Пословная токенизация с LSTM


In [ ]:
# Настройка LSTM для пословной генерации
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("Настраиваем LSTM для пословной генерации...")

# Токенизация слов
tokenizer = Tokenizer()
tokenizer.fit_on_texts([full_cleaned_text])
word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}
total_words = len(word_index) + 1

print(f"Общий словарь слов: {total_words}")

# Создание последовательностей слов
input_sequences = []
words = full_cleaned_text.split()

for i in range(1, len(words)):
    ngram = words[max(0, i-5):i+1]
    encoded = tokenizer.texts_to_sequences([" ".join(ngram)])[0]
    if len(encoded) >= 2:
        input_sequences.append(encoded)

max_len = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_len)

X_words = input_sequences[:, :-1]
y_words = to_categorical(input_sequences[:, -1], num_classes=total_words)

print(f"Размер обучающих данных: {X_words.shape}")
print(f"Размер целевых данных: {y_words.shape}")

# Создание однослойной LSTM для слов
model_words_1 = Sequential([
    tf.keras.layers.Embedding(total_words, 64, input_length=X_words.shape[1]),
    LSTM(128),
    Dense(total_words, activation='softmax')
])

model_words_1.compile(loss='categorical_crossentropy', optimizer='adam')

print("Архитектура однослойной LSTM для слов:")
model_words_1.summary()

# Обучение модели
print("Начинаем обучение однослойной LSTM для слов...")
model_words_1.fit(X_words, y_words, epochs=100, batch_size=128)

print("Обучение завершено!")


In [ ]:
def generate_text_word_lstm(model, seed_text, num_words=10, temperature=1.0):
    """
    Функция генерации текста по словам с использованием LSTM
    
    Args:
        model: Обученная LSTM модель
        seed_text (str): Начальный текст
        num_words (int): Количество слов для генерации
        temperature (float): Температура для сэмплирования
        
    Returns:
        str: Сгенерированный текст
    """
    for _ in range(num_words):
        # Токенизируем текущий текст
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=X_words.shape[1])

        # Получаем предсказания
        preds = model.predict(token_list, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        # Выбираем следующее слово
        next_index = np.random.choice(range(total_words), p=preds)
        next_word = index_word.get(next_index, "")
        seed_text += " " + next_word
    
    return seed_text

# Тестируем генерацию слов
print("Генерация текста по словам с LSTM:")
print("Начальная фраза: 'волшебная палочка'")
generated_words = generate_text_word_lstm(model_words_1, "волшебная палочка")
print(generated_words)

print("\nГенерация с другой фразой:")
print("Начальная фраза: 'малфой'")
generated_words2 = generate_text_word_lstm(model_words_1, "малфой")
print(generated_words2)


## BPE (Byte Pair Encoding) токенизация


In [ ]:
# Установка библиотеки tokenizers для BPE
import subprocess
import sys

try:
    from tokenizers import Tokenizer, models, trainers, pre_tokenizers
    print("Библиотека tokenizers уже установлена")
except ImportError:
    print("Устанавливаем библиотеку tokenizers...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tokenizers"])
    from tokenizers import Tokenizer, models, trainers, pre_tokenizers

from tensorflow.keras.utils import to_categorical

print("Настраиваем BPE токенизацию...")

# Создаем и обучаем BPE токенизатор
bpe_tokenizer = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(special_tokens=["<PAD>"])
bpe_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
bpe_tokenizer.train_from_iterator(full_cleaned_text.splitlines(), trainer)

print("BPE токенизатор обучен!")

# Создание последовательностей с BPE токенизацией
tokens = bpe_tokenizer.encode(full_cleaned_text).ids
seq_len = 40
X_bpe, y_bpe = [], []

for i in range(seq_len, len(tokens)):
    X_bpe.append(tokens[i-seq_len:i])
    y_bpe.append(tokens[i])

vocab_size_bpe = bpe_tokenizer.get_vocab_size()

X_bpe = np.array(X_bpe)
y_bpe = to_categorical(y_bpe, num_classes=vocab_size_bpe)

print(f"Размер словаря BPE: {vocab_size_bpe}")
print(f"Размер обучающих данных: {X_bpe.shape}")
print(f"Размер целевых данных: {y_bpe.shape}")


### Однослойная LSTM с BPE


In [ ]:
# Создание однослойной LSTM модели с BPE
model_bpe_1 = Sequential([
    tf.keras.layers.Embedding(vocab_size_bpe, 64, input_length=seq_len),
    LSTM(128),
    Dense(vocab_size_bpe, activation='softmax')
])

model_bpe_1.compile(loss='categorical_crossentropy', optimizer='adam')

print("Архитектура однослойной LSTM с BPE:")
model_bpe_1.summary()

# Обучение модели
print("Начинаем обучение однослойной LSTM с BPE...")
model_bpe_1.fit(X_bpe, y_bpe, epochs=150, batch_size=128)

print("Обучение завершено!")

# Функция генерации с BPE
def generate_text_bpe_lstm(model, tokenizer_bpe, seed_text, gen_tokens=50, temperature=1.0):
    """
    Функция генерации текста с использованием BPE токенизации
    
    Args:
        model: Обученная модель
        tokenizer_bpe: BPE токенизатор
        seed_text (str): Начальный текст
        gen_tokens (int): Количество токенов для генерации
        temperature (float): Температура сэмплирования
        
    Returns:
        str: Сгенерированный текст
    """
    tokens = tokenizer_bpe.encode(seed_text).ids
    generated = tokens[:]

    for _ in range(gen_tokens):
        input_seq = generated[-seq_len:]
        input_seq = pad_sequences([input_seq], maxlen=seq_len)

        preds = model.predict(input_seq, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        next_index = np.random.choice(range(vocab_size_bpe), p=preds)
        generated.append(next_index)

    decoded = tokenizer_bpe.decode(generated)
    return decoded

# Тестируем генерацию с BPE
print("Генерация текста с BPE токенизацией:")
print("Начальная фраза: 'заклинание против'")
generated_bpe = generate_text_bpe_lstm(model_bpe_1, bpe_tokenizer, "заклинание против")
print(generated_bpe)

print("\nГенерация с другой фразой:")
print("Начальная фраза: 'проклятые маглы'")
generated_bpe2 = generate_text_bpe_lstm(model_bpe_1, bpe_tokenizer, "проклятые маглы")
print(generated_bpe2)


# Часть 3: Двунаправленная LSTM


## Посимвольная токенизация с Bidirectional LSTM


In [ ]:
# Импорт библиотек для Bidirectional LSTM
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

print("Настраиваем Bidirectional LSTM для посимвольной генерации...")

# Подготовка данных (аналогично предыдущим моделям)
chars = sorted(set(full_cleaned_text))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for c, i in char2idx.items()}
vocab_size = len(chars)

# Создание последовательностей
seq_length = 40
step = 3
input_seqs, target_chars = [], []

for i in range(0, len(full_cleaned_text) - seq_length, step):
    seq = full_cleaned_text[i:i + seq_length]
    target = full_cleaned_text[i + seq_length]
    input_seqs.append([char2idx[c] for c in seq])
    target_chars.append(char2idx[target])

X_char = tf.keras.utils.to_categorical(input_seqs, num_classes=vocab_size)
y_char = tf.keras.utils.to_categorical(target_chars, num_classes=vocab_size)

print(f"Размер обучающих данных: {X_char.shape}")
print(f"Размер целевых данных: {y_char.shape}")

# Создание модели Bidirectional LSTM
model_char = Sequential([
    Bidirectional(LSTM(128), input_shape=(seq_length, vocab_size)),
    Dense(vocab_size, activation='softmax')
])

model_char.compile(loss='categorical_crossentropy', optimizer='adam')

print("Архитектура Bidirectional LSTM:")
model_char.summary()

# Обучение модели
print("Начинаем обучение Bidirectional LSTM...")
model_char.fit(X_char, y_char, batch_size=128, epochs=50)

print("Обучение завершено!")

# Функция генерации для Bidirectional LSTM
def generate_char_text(model, seed_text="гарри", length=20, temperature=1.0):
    """
    Функция генерации текста с Bidirectional LSTM
    
    Args:
        model: Обученная модель
        seed_text (str): Начальный текст
        length (int): Длина генерируемого текста
        temperature (float): Температура сэмплирования
        
    Returns:
        str: Сгенерированный текст
    """
    generated = seed_text
    
    for _ in range(length):
        input_seq = [char2idx.get(c, 0) for c in generated[-seq_length:]]
        input_seq = tf.keras.utils.to_categorical(input_seq, num_classes=vocab_size)
        input_seq = np.expand_dims(input_seq, axis=0)

        preds = model.predict(input_seq, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        preds = np.exp(preds) / np.sum(np.exp(preds))

        next_idx = np.random.choice(range(vocab_size), p=preds)
        next_char = idx2char[next_idx]
        generated += next_char
    
    return generated

# Тестируем генерацию
print("Генерация текста с Bidirectional LSTM:")
print("Начальное слово: 'заклинание'")
generated_char = generate_char_text(model_char, "заклинание", length=60)
print(generated_char)


# Часть 4: GPT с нуля


## Символьная GPT модель


In [ ]:
# Импорт библиотек для Transformer архитектуры
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

print("Создаем GPT-style модель с нуля...")

# Подготовка данных для символов
vocab = sorted(set(full_cleaned_text))
char2idx = {c: i for i, c in enumerate(vocab)}
idx2char = {i: c for c, i in char2idx.items()}
vocab_size = len(vocab)

print(f"Размер словаря символов: {vocab_size}")

# Преобразование в числовой вид
encoded = [char2idx[c] for c in full_cleaned_text]

# Создание обучающих пар
context_len = 64
X, y = [], []
for i in range(len(encoded) - context_len):
    X.append(encoded[i:i + context_len])
    y.append(encoded[i + context_len])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)

print(f"Размер входных данных: {X.shape}")
print(f"Размер целевых данных: {y.shape}")

# Определение кастомных слоев для Transformer
from tensorflow.keras.layers import Layer, Dense, LayerNormalization
import tensorflow.keras.backend as K

class MaskedSelfAttention(Layer):
    """
    Слой маскированного self-attention для GPT архитектуры
    """
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.proj_q = Dense(embed_dim)
        self.proj_k = Dense(embed_dim)
        self.proj_v = Dense(embed_dim)
        self.out = Dense(embed_dim)

    def call(self, x):
        B, T, C = tf.shape(x)[0], tf.shape(x)[1], self.embed_dim
        q = self.proj_q(x)
        k = self.proj_k(x)
        v = self.proj_v(x)

        # Разделение на головы внимания
        q = tf.concat(tf.split(q, self.num_heads, axis=-1), axis=0)
        k = tf.concat(tf.split(k, self.num_heads, axis=-1), axis=0)
        v = tf.concat(tf.split(v, self.num_heads, axis=-1), axis=0)

        # Вычисление attention scores
        att = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(C // self.num_heads, tf.float32))

        # Маскирование будущих позиций (causal mask)
        mask = tf.linalg.band_part(tf.ones((T, T)), -1, 0)
        att = tf.where(mask == 0, -1e10, att)

        att = tf.nn.softmax(att, axis=-1)
        out = tf.matmul(att, v)
        out = tf.concat(tf.split(out, self.num_heads, axis=0), axis=-1)
        return self.out(out)

class TransformerBlock(Layer):
    """
    Блок Transformer с self-attention и feed-forward сетью
    """
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.att = MaskedSelfAttention(embed_dim, num_heads)
        self.norm1 = LayerNormalization()
        self.ff = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embed_dim)
        ])
        self.norm2 = LayerNormalization()

    def call(self, x):
        # Self-attention с residual connection
        x = x + self.att(self.norm1(x))
        # Feed-forward с residual connection
        x = x + self.ff(self.norm2(x))
        return x

class PositionalEncoding(Layer):
    """
    Позиционное кодирование для Transformer
    """
    def __init__(self, max_len, embed_dim):
        super().__init__()
        pos = np.arange(max_len)[:, None]
        i = np.arange(embed_dim)[None, :]
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(embed_dim))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.constant(angle_rads[np.newaxis, ...], dtype=tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

print("Кастомные слои Transformer определены!")


In [ ]:
# Создание GPT модели
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding

# Параметры модели
embed_dim = 128
num_heads = 4
ff_dim = 256
num_layers = 4

print(f"Создаем GPT модель с параметрами:")
print(f"- Размерность эмбеддинга: {embed_dim}")
print(f"- Количество голов внимания: {num_heads}")
print(f"- Размерность feed-forward: {ff_dim}")
print(f"- Количество слоев: {num_layers}")

# Создание модели
inp = Input(shape=(context_len,))
x = Embedding(vocab_size, embed_dim)(inp)
x = PositionalEncoding(context_len, embed_dim)(x)

# Добавляем блоки Transformer
for i in range(num_layers):
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

# Выходной слой для предсказания следующего токена
out = Dense(vocab_size, activation="softmax")(x[:, -1, :])

gpt_model = Model(inputs=inp, outputs=out)
gpt_model.compile(loss="categorical_crossentropy", optimizer="adam")

print("Архитектура GPT модели:")
gpt_model.summary()

# Обучение модели
print("Начинаем обучение GPT модели...")
print("Внимание: Обучение может занять значительное время!")

gpt_model.fit(X, y, batch_size=128, epochs=10)

print("Обучение GPT модели завершено!")


In [ ]:
def generate_gpt(seed_text, length=300, temperature=1.0):
    """
    Функция генерации текста с помощью GPT модели
    
    Args:
        seed_text (str): Начальный текст
        length (int): Длина генерируемого текста
        temperature (float): Температура сэмплирования
        
    Returns:
        str: Сгенерированный текст
    """
    generated = seed_text
    context = [char2idx.get(c, 0) for c in seed_text][-context_len:]

    for _ in range(length):
        # Подготавливаем входную последовательность
        padded = pad_sequences([context], maxlen=context_len)
        preds = gpt_model.predict(padded, verbose=0)[0]

        # Безопасное масштабирование вероятностей
        preds = np.asarray(preds).astype("float64")
        preds = np.log(np.clip(preds, 1e-8, 1.0)) / temperature
        preds = np.exp(preds)
        preds = preds / np.sum(preds)

        # Выбираем следующий символ
        next_idx = np.random.choice(len(preds), p=preds)
        next_char = idx2char[next_idx]

        generated += next_char
        context.append(next_idx)
        context = context[-context_len:]  # Сохраняем только последние context_len символов

    return generated

# Тестируем генерацию с GPT
print("Генерация текста с GPT моделью:")
print("Начальное слово: 'гарри'")
generated_gpt = generate_gpt("гарри ", 300, temperature=1.0)
print(generated_gpt)

print("\nГенерация с другим начальным словом:")
print("Начальное слово: 'гарри' (короткая генерация)")
generated_gpt_short = generate_gpt("гарри ", 40)
print(generated_gpt_short)


# Дообучение готовой GPT модели


In [ ]:
# Установка библиотек для дообучения
try:
    from datasets import Dataset
    from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
    print("Библиотеки transformers и datasets уже установлены")
except ImportError:
    print("Устанавливаем необходимые библиотеки...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers", "datasets"])
    from datasets import Dataset
    from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer

print("Настраиваем дообучение готовой GPT модели...")

# Создание датасета из наших данных
dataset = Dataset.from_pandas(df[["text"]])

print(f"Создан датасет с {len(dataset)} записями")

# Загрузка предобученного токенизатора для русского языка
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    """
    Функция токенизации для датасета
    """
    encodings = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()
    return encodings

# Токенизация датасета
tokenized = dataset.map(tokenize_fn, batched=True)

print("Датасет токенизирован!")

# Загрузка предобученной модели
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model.resize_token_embeddings(len(tokenizer))

print("Предобученная модель загружена!")
print(f"Размер модели: {model.num_parameters():,} параметров")


In [ ]:
# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir="./rugpt-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    prediction_loss_only=True,
)

# Создание тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
)

print("Начинаем дообучение модели...")
print("Это может занять значительное время!")

# Запуск обучения
trainer.train()

print("Дообучение завершено!")

# Создание pipeline для генерации
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

print("Pipeline для генерации создан!")

# Тестируем генерацию с дообученной моделью
print("Генерация текста с дообученной моделью:")
print("Начальная фраза: 'в хогвартсе'")
generated_finetuned = generator("в хогвартсе", max_length=100, do_sample=True, temperature=0.9)
print(generated_finetuned[0]['generated_text'])

print("\nГенерация с другой фразой:")
print("Начальная фраза: 'гермиона сказала'")
generated_finetuned2 = generator("гермиона сказала", max_length=100, do_sample=True, temperature=1.0)
print(generated_finetuned2[0]['generated_text'])


# Заключение и сравнение моделей

## Анализ результатов генерации

В данной лабораторной работе мы реализовали и протестировали различные подходы к генерации текста:

### 1. Simple RNN
- **Преимущества**: Простота реализации, быстрая тренировка
- **Недостатки**: Проблема исчезающего градиента, плохое качество генерации
- **Применение**: Базовый уровень для понимания принципов работы

### 2. LSTM (однонаправленная и многослойная)
- **Преимущества**: Решение проблемы долгосрочных зависимостей, лучшее качество генерации
- **Недостатки**: Медленная тренировка, ограниченная контекстная память
- **Применение**: Хорошо подходит для последовательностей средней длины

### 3. Bidirectional LSTM
- **Преимущества**: Двунаправленная обработка контекста
- **Недостатки**: Не подходит для генерации в реальном времени
- **Применение**: Анализ и понимание текста, но не генерация

### 4. GPT-style Transformer
- **Преимущества**: Параллельная обработка, механизм внимания, высокое качество
- **Недостатки**: Требует много вычислительных ресурсов
- **Применение**: Современный стандарт для генерации текста

### 5. Дообучение готовых моделей
- **Преимущества**: Использование предобученных знаний, быстрое дообучение
- **Недостатки**: Зависимость от качества предобученной модели
- **Применение**: Практический подход для реальных задач

## Рекомендации по выбору архитектуры

- **Для изучения**: Начните с Simple RNN, затем переходите к LSTM
- **Для экспериментов**: Используйте LSTM с различными токенизациями
- **Для производства**: Рассмотрите Transformer архитектуры или дообучение готовых моделей
- **Для русского языка**: Предпочтительно использовать предобученные модели с дообучением

## Дальнейшее развитие

1. **Увеличение размера модели**: Больше параметров = лучшее качество
2. **Улучшение токенизации**: BPE, SentencePiece, современные методы
3. **Оптимизация обучения**: Learning rate scheduling, gradient clipping
4. **Оценка качества**: BLEU, ROUGE, человеческая оценка
5. **Специализация**: Модели для конкретных доменов или стилей
